In [17]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:99% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:24pt;}
div.text_cell_render.rendered_html{font-size:20pt;}
div.text_cell_render ul li, div.text_cell_render ol li p, code{font-size:22pt; line-height:30px;}
div.output {font-size:24pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:24pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:24pt;padding:5px;}
table.dataframe{font-size:24px;}
</style>
"""))

# <span style="color:red">ch2.Ollama_LLM활용의 기본 개념(LangChain)</span>
# 1.  LLM을 활용하여 답변 생성하기
## 1) Ollama를 이용한 로컬 LLM 이용
성능은 GPT, CLaude 같은 모델보다 떨어지나, 개념설명을 위해 open source 모델 사용

### ollama.com 다운로드 -> 설치 -> 모델 pull
- cmd창이나 powershell 창에 ollama pull deepseek-r1:1.5b
https://docs.langchain.com/oss/python/integrations/chat/ollama

In [5]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="deepseek-r1:1.5b")
result = llm.invoke("What is the captital of Korea?")
result

AIMessage(content='The capital of Korea is Seoul.', additional_kwargs={}, response_metadata={'model': 'deepseek-r1:1.5b', 'created_at': '2025-12-09T02:05:09.771942Z', 'done': True, 'done_reason': 'stop', 'total_duration': 311237600, 'load_duration': 100486800, 'prompt_eval_count': 11, 'prompt_eval_duration': 15993300, 'eval_count': 12, 'eval_duration': 175396000, 'logprobs': None, 'model_name': 'deepseek-r1:1.5b', 'model_provider': 'ollama'}, id='lc_run--019b00da-f9d3-7992-a202-21a1966fa175-0', usage_metadata={'input_tokens': 11, 'output_tokens': 12, 'total_tokens': 23})

### 모델 pull
- cmd창이나 powershell창(window키+R에서 powershell)에서 ollama pull llama3.2:1b

In [13]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
result = llm.invoke("What is the capital of Korea?")
result

AIMessage(content="The capital of South Korea is Seoul. However, it's worth noting that there are also two other cities that serve as major centers: Busan in the south and Incheon in the west.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T02:22:50.7896232Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1154751300, 'load_duration': 125474600, 'prompt_eval_count': 32, 'prompt_eval_duration': 39557100, 'eval_count': 40, 'eval_duration': 939153800, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b00eb-2722-7dc1-b4f3-2a262bfc78fe-0', usage_metadata={'input_tokens': 32, 'output_tokens': 40, 'total_tokens': 72})

In [14]:
result.content

"The capital of South Korea is Seoul. However, it's worth noting that there are also two other cities that serve as major centers: Busan in the south and Incheon in the west."

In [15]:
result = llm.invoke("일본 수도가 어디예요?")
result.content

'Japan의 수도는 Tokyo입니다. Tokyoは日本最も나icing도高い都市です。'

## 2) openai 활용
- pip install langchain-openai
- https://auth.openai.com/log-in

In [3]:
# 환경변수 가져오기
from dotenv import load_dotenv
import os
load_dotenv()
# os.getenv("OPENAI_API_KEY")

True

In [4]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-5-nano",
                #api_key=os.getenv("OPENAI_API_KEY")
                )
result = llm.invoke("What is the capitaal of Korea? Return the name of the city only.")

In [5]:
result

AIMessage(content='Seoul', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 203, 'prompt_tokens': 22, 'total_tokens': 225, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CkkVef1qFLb60opVXts7a5APihm3W', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b0192-050b-7370-a2e9-717f6b021437-0', usage_metadata={'input_tokens': 22, 'output_tokens': 203, 'total_tokens': 225, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}})

In [7]:
# Azure : OPENAI_API_VERSION키값
# from langchain_openai import AzureOpenAI
# llm = AzureOpenAI(model="gpt-5-nano")

# 2. 렝체인 스타일로 프롬프트 작성
- 프롬프트 : llm호출시 쓰는 질문

In [10]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model="llama3.2:1b")
# llm.invoke(0)
# PromptValue, str, BaseMessages리스트

## 1) 기본 프롬프트 템플릿 사용
- PromptTemplate을 사용하여 변수가 포함된 템플릿을 작성하면 PromptValue

In [16]:
from langchain_core.prompts import PromptTemplate
llm = ChatOllama(model="llama3.2:1b")
prompt_template = PromptTemplate(
    template="What is the capital of {country}?", # {}안의 값을 새로운 값으로 대입 가능
    input_variables = ["country"]
)
# country = input('어느 나라의 수도를 알고 싶으신가요?')
prompt = prompt_template.invoke({"country":"Korea"})
print(prompt)
llm.invoke(prompt)

text='What is the capital of Korea?'


AIMessage(content="The capital of South Korea is Seoul. However, it's worth noting that North Korea also claims Seoul as its own capital, and the two countries have a long-standing rivalry. But for official purposes, Seoul is recognized as the capital of South Korea.", additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T05:47:44.6780933Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2659583300, 'load_duration': 1885352500, 'prompt_eval_count': 32, 'prompt_eval_duration': 40899100, 'eval_count': 51, 'eval_duration': 671917900, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01a6-b841-7863-84ba-f4047df89196-0', usage_metadata={'input_tokens': 32, 'output_tokens': 51, 'total_tokens': 83})

## 2) 메세지 기반 프롬프트 작성
- BaseMessage 리스트
- BaseMessage 상속받은 클래스 : AIMessage, HumanMessage, SystemMessage, ToolMessage
- vscode에서 ctrl+shift+p : python:select interpreter입력 -> python환경선택
- vscode에서 커널 선택

In [18]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범답안
    HumanMessage(content="What is the capital of France?"), # 모범질문
    AIMessage(content="The capital of France is Paris."),    # 모범답안
    HumanMessage(content="What is the capital of Korea?")
]
llm.invoke(message_list)

AIMessage(content='The capital of South Korea is Seoul. The capital of North Korea is Pyongyang.', additional_kwargs={}, response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-12-09T06:04:19.061392Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2441705100, 'load_duration': 1911786300, 'prompt_eval_count': 86, 'prompt_eval_duration': 81113100, 'eval_count': 17, 'eval_duration': 434927200, 'logprobs': None, 'model_name': 'llama3.2:1b', 'model_provider': 'ollama'}, id='lc_run--019b01b5-e569-7e32-9efe-c3ca13a09ae6-0', usage_metadata={'input_tokens': 86, 'output_tokens': 17, 'total_tokens': 103})

In [19]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
message_list = [
    SystemMessage(content="You are a helpful assistant!"), # 페르소나 부여
    HumanMessage(content="What is the capital of Italy?"), # 모범질문
    AIMessage(content="The capital of Italy is Rome."),    # 모범답안
    HumanMessage(content="What is the capital of France?"), # 모범질문
    AIMessage(content="The capital of France is Paris."),    # 모범답안
    HumanMessage(content="What is the capital of {country}?")
]
print(message_list)

[SystemMessage(content='You are a helpful assistant!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Italy?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of Italy is Rome.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of {country}?', additional_kwargs={}, response_metadata={})]


## 3) ChatPromptTemplate 사용
- BaseMessage리스트 -> 튜플리스트

In [20]:
# 위의 BaseMessage 리스트를 수정
# PromptTemplate : 프롬프트에 변수포함, 
# ChatPromptTemplate : SystemPrompt설정(페르소나), few shot설정, 변수포함
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
chatPrompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpfull assistant!"),
    ('human', "What is the capital of Italy?"), # 모범질문
    ("ai", "The capital of Italy is Rome."),    # 모범답안
    ('human', "What is the capital of France?"), # 모범질문
    ("ai", "The capital of France is Paris."),    # 모범답안
    ("human", "What is the capital of {country}?")
])
country = input("어느 나라 수도가 궁금하세요")
prompt = chatPrompt_template.invoke({"country": country})
print("프롬프트 : ", prompt, type(prompt))

어느 나라 수도가 궁금하세요Korea
프롬프트 :  messages=[SystemMessage(content='You are a helpfull assistant!', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Italy?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of Italy is Rome.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of France?', additional_kwargs={}, response_metadata={}), AIMessage(content='The capital of France is Paris.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is the capital of Korea?', additional_kwargs={}, response_metadata={})] <class 'langchain_core.prompt_values.ChatPromptValue'>
